[Modern Parallel and Distributed Python: 
A Quick Tutorial on Ray](https://rise.cs.berkeley.edu/blog/modern-parallel-and-distributed-python-a-quick-tutorial-on-ray/)

ROBERT NISHIHARA, FEBRUARY 11, 2019



low-level primitives for sending/receiving msg:
* OpenMPI
* Python multiprocessing
* ZeroMQ

domain-specific tools:
* TensorFlow for model training, 
* Spark for data processing and SQL, 
* Flink for stream processing

Ray occupies a unique middle ground. Instead of introducing new concepts. Ray takes the existing concepts of **functions** and **classes** and translates them to the distributed setting as **tasks** and **actors**. This API choice allows serial applications to be parallelized without major modifications.

To turn a Python function f into a “remote function” (a function that can be executed remotely and asynchronously), we declare the function with the @ray.remote decorator. Then function invocations via f.remote() will immediately return futures (a future is a reference to the eventual output), and the actual function execution will take place in the background (we refer to this execution as a task).

### From Functions to Tasks

In [2]:
import ray
import time

# shutdown Ray
#ray.shutdown()

# Start Ray.
ray.init(ignore_reinit_error=True)

# decorate a function for remote execution
@ray.remote
def f(x):
    time.sleep(1)
    return x, time.time()

# Start 4 tasks in parallel.
task_ids = []
for i in range(4):
    task_ids.append(f.remote(i))
    
# Wait for the tasks to complete and retrieve the results.
# With at least 4 cores, this will take 1 second.
results = ray.get(task_ids)  # [0, 1, 2, 3]

print(results)

2019-08-18 10:57:19,251	ERROR worker.py:1379 -- Calling ray.init() again after it has already been called.


[(0, 1566140240.2645104), (1, 1566140240.266269), (2, 1566140240.2657173), (3, 1566140240.2644246)]


In [3]:
for i in results:
    print(i)

(0, 1566140240.2645104)
(1, 1566140240.266269)
(2, 1566140240.2657173)
(3, 1566140240.2644246)


##### Task dependencies

In [4]:
import numpy as np

@ray.remote
def create_matrix(size):
    return np.random.normal(size=size)

@ray.remote
def multiply_matrices(x, y):
    return np.dot(x, y)

x_id = create_matrix.remote([1000, 1000])
y_id = create_matrix.remote([1000, 1000])
z_id = multiply_matrices.remote(x_id, y_id)

# Get the results.
z = ray.get(z_id)
z

array([[-19.08510069,  43.13664083, -34.31825234, ..., -14.36481326,
         14.19681818, -45.26217878],
       [ -1.56324961,  52.23076381, -19.55718068, ..., -33.41550062,
          4.24226204,  11.05515945],
       [ 10.56462708,  38.07884699, -23.06211695, ..., -19.94282994,
         35.09478634,   9.0282199 ],
       ...,
       [-12.02707733, -32.79849117,  32.38071694, ...,  10.82940165,
        -37.6186476 ,  15.22729844],
       [  7.76369977, -14.30707451, -10.87264475, ..., -47.1691431 ,
         -2.69797863,  10.78312351],
       [-51.69958277,  -0.20767613,   6.4782947 , ...,   7.42094039,
         50.69998715, -37.6367722 ]])

In [5]:
z.shape

(1000, 1000)

In [6]:
x_id, y_id, z_id

(ObjectID(28db05a598a1e7eacd5d965b90d664eb01000000),
 ObjectID(9b598c91bd9f9703657015a5302eeb5101000000),
 ObjectID(91a79a584cdc65d65e4aea8f58d3bc5701000000))

##### Aggregating Values Efficiently

In [19]:
import time

@ray.remote
def add(x, y=None):
    time.sleep(1)
    return x + (y if y else 0)

In [20]:
%%time
# Aggregate the values slowly. This approach takes O(n) where n is the
# number of values being aggregated. In this case, 7 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(id1, 3)
id3 = add.remote(id2, 4)
id4 = add.remote(id3, 5)
id5 = add.remote(id4, 6)
id6 = add.remote(id5, 7)
id7 = add.remote(id6, 8)
result = ray.get(id7)
print(result)

36
CPU times: user 255 ms, sys: 85.2 ms, total: 340 ms
Wall time: 7.04 s


In [21]:
%%time
# Aggregate the values in a tree-structured pattern. This approach
# takes O(log(n)). In this case, 3 seconds.
id1 = add.remote(1, 2)
id2 = add.remote(3, 4)
id3 = add.remote(5, 6)
id4 = add.remote(7, 8)
id5 = add.remote(id1, id2)
id6 = add.remote(id3, id4)
id7 = add.remote(id5, id6)
result = ray.get(id7)
print(result)

36
CPU times: user 109 ms, sys: 36.3 ms, total: 145 ms
Wall time: 3.01 s


In [26]:
%%time
# Slow approach.
values = [1, 2, 3, 4, 5, 6, 7, 8, 9]
while len(values) > 1:
    values = [add.remote(values[0], values[1])] + values[2:]
result = ray.get(values[0])
print(result)

45
CPU times: user 317 ms, sys: 69 ms, total: 386 ms
Wall time: 8.03 s


In [27]:
%%time
# Fast approach.
values = [1, 2, 3, 4, 5, 6, 7, 8, 9]
while len(values) > 1:
    values = values[2:] + [add.remote(values[0], values[1])]
result = ray.get(values[0])
print(result)

45
CPU times: user 149 ms, sys: 46.7 ms, total: 195 ms
Wall time: 4.02 s


### From Classes to Actors

In [46]:
@ray.remote
class Adder(object):
    def __init__(self, x=None, y=None):
        self.x = x if x else 0
        self.y = y if y else 0
        self.sum = self.x + self.y

    def clear(self):
        self.x = 0
        self.y = 0
        self.sum = self.x + self.y
        
    def get_sum(self):
        return self.sum
    
    def add(self, x=None, y=None):
        self.x = x if x else 0
        self.y = y if y else 0
        self.sum += self.x + self.y
        return self.sum
    
# Create an actor process.
a = Adder.remote()

# Check the actor's counter value.
print(ray.get(a.get_sum.remote()))  

# Increment the counter twice and check the value again.
for _ in range(10):
    a.add.remote(_)

print(ray.get(a.get_sum.remote()))  # 45

0
45


In [93]:
# %%time  # this interfere with results

ray.init(ignore_reinit_error=True)
start_time = time.time()
n_actors = 3
n_numbers = 1000
actors = []
for i in range(n_actors):
    actors.append(Adder.remote())

for n in range(n_numbers):
    i = n % n_actors
    actors[i].add.remote(n)
    
result = sum(ray.get([actors[i].get_sum.remote() for i in range(n_actors)]))
stop_time = time.time()
print(f"result={result} in {stop_time - start_time} sec [actors={n_actors}]")


2019-08-18 12:12:35,357	ERROR worker.py:1379 -- Calling ray.init() again after it has already been called.


result=499500 in 1.304793119430542 sec [actors=3]


result=499500 in 4.087217569351196 sec [actors=10]

result=499500 in 2.583878993988037 sec [actors=5]

result=499500 in 1.0627317428588867 sec [actors=2]

In [28]:
@ray.remote
class Counter(object):
    def __init__(self, x):
        self.x = x
    
    def inc(self):
        self.x += 1
    
    def get_value(self):
        return self.x

# Create an actor process.
c = Counter.remote(5)

# Check the actor's counter value.
print(ray.get(c.get_value.remote()))  # 5

# Increment the counter twice and check the value again.
for _ in range(4):
    c.inc.remote()

print(ray.get(c.get_value.remote()))  # 9

5
9


#### Actor Handles

One of the most powerful aspects of actors is that we can pass around handles to an actor, which allows other actors or other tasks to all invoke methods on the same actor.

The following example creates an actor that stores messages. Several worker tasks repeatedly push messages to the actor, and the main Python script reads the messages periodically.

In [21]:
import time


@ray.remote
class MessageActor(object):
    def __init__(self):
        self.messages = []
    
    def add_message(self, message):
        self.messages.append(message)
    
    def get_and_clear_messages(self):
        messages = self.messages
        self.messages = []
        return messages


# Define a remote function which loops around and pushes
# messages to the actor.
@ray.remote
def worker(message_actor, j):
    for i in range(10):
        time.sleep(1)
        message_actor.add_message.remote(
            "MSG {} by WKR {}.".format(i, j))


# Create a message actor.
message_actor = MessageActor.remote()

# Start 3 tasks that push messages to the actor.
[worker.remote(message_actor, j) for j in range(3)]

# Periodically get the messages and print them.
for _ in range(10):
    new_messages = ray.get(message_actor.get_and_clear_messages.remote())
    print("New MSG:\n", new_messages)
    time.sleep(1)

# This script prints something like the following:
# New messages: []
# New messages: ['Message 0 from worker 1.', 'Message 0 from worker 0.']
# New messages: ['Message 0 from worker 2.', 'Message 1 from worker 1.', 'Message 1 from worker 0.', 'Message 1 from worker 2.']
# New messages: ['Message 2 from worker 1.', 'Message 2 from worker 0.', 'Message 2 from worker 2.']
# New messages: ['Message 3 from worker 2.', 'Message 3 from worker 1.', 'Message 3 from worker 0.']
# New messages: ['Message 4 from worker 2.', 'Message 4 from worker 0.', 'Message 4 from worker 1.']
# New messages: ['Message 5 from worker 2.', 'Message 5 from worker 0.', 'Message 5 from worker 1.']

New MSG:
 []
New MSG:
 ['MSG 0 by WKR 0.', 'MSG 0 by WKR 1.']
New MSG:
 ['MSG 1 by WKR 0.', 'MSG 1 by WKR 1.']
New MSG:
 ['MSG 0 by WKR 2.', 'MSG 2 by WKR 0.', 'MSG 2 by WKR 1.']
New MSG:
 ['MSG 1 by WKR 2.', 'MSG 3 by WKR 0.', 'MSG 3 by WKR 1.']
New MSG:
 ['MSG 2 by WKR 2.', 'MSG 4 by WKR 0.', 'MSG 4 by WKR 1.']
New MSG:
 ['MSG 3 by WKR 2.', 'MSG 5 by WKR 0.', 'MSG 5 by WKR 1.']
New MSG:
 ['MSG 4 by WKR 2.', 'MSG 6 by WKR 0.', 'MSG 6 by WKR 1.']
New MSG:
 ['MSG 5 by WKR 2.', 'MSG 7 by WKR 0.', 'MSG 7 by WKR 1.']
New MSG:
 ['MSG 6 by WKR 2.', 'MSG 8 by WKR 0.', 'MSG 8 by WKR 1.']
